In [61]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn.functional as F
from PIL import Image
import os

In [62]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [63]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(),
    transforms.ToTensor()
])

In [64]:
lr =0.001
batch_size=100
epochs = 10

In [65]:
# Load train and test data
train_dir = r'C:\PetImages'
import glob

#train_list = glob.glob(os.path.join(train_dir,'*.jpg')) # LINUX
train_list = []

for path, currentDirectory, files in os.walk(train_dir):
    for file in files:
        train_list.append(os.path.join(path, file))


In [66]:
class dataset(torch.utils.data.Dataset):
    def __init__(self,file_list,transform = None):
        self.file_list=file_list
        self.transform=transform

    def __len__(self):
        self.filelength =len(self.file_list)
        return self.filelength

    def __getitem__(self,idx):
        img_path =self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)

        label = os.path.split(os.path.split(img_path)[-2])[-1]
        if label == 'Dog':
            label = torch.tensor(1)
        elif label == 'Cat':
            label = torch.tensor(0)

        return img_transformed,label

In [67]:
from sklearn.model_selection import train_test_split
train_list,val_list = train_test_split(train_list , test_size =0.2)

In [68]:
train_data = dataset(train_list,transform=transform)
val_data = dataset(val_list,transform=transform)

In [69]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size,shuffle = True)
val_loader = torch.utils.data.DataLoader(dataset = val_data, batch_size = batch_size,shuffle = True)

In [70]:
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn,self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=3,padding=0,stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size=3,padding=0,stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)

        )


        self.layer3 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3,padding=0,stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)

        )

        self.fc1 = nn.Linear(3*3*64,10)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(10,2)
        self.relu = nn.ReLU()

    def forward(self,x):
        out =self.layer1(x)
        out =self.layer2(out)
        out =self.layer3(out)
        out =out.view(out.size(0),-1)
        out =self.relu(self.fc1(out))
        out =self.fc2(out)
        return out


In [71]:
model = Cnn().to(device)
model.train()
optimizer = optim.Adam(params = model.parameters(),lr =0.001)
criterion = nn.CrossEntropyLoss()

In [72]:
epochs = 10

for epoch in range(epochs):
    epoch_loss =0
    epoch_accuracy = 0

    for data,label in train_loader:
        data= data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output,label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = ((output.argmax(dim=1)==label).float().mean())
        epoch_accuracy += acc/len(train_loader)
        epoch_loss += loss/len(train_loader)

    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch+1, epoch_accuracy,epoch_loss))

    with torch.no_grad():
        epoch_val_accuracy =0
        epoch_val_loss = 0
        for data,label in  val_loader:
            data= data.to(device)

            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output,label)


            acc = ((val_output.argmax(dim=1)==label).float().mean())
            epoch_val_accuracy += acc/len(val_loader)
            epoch_val_loss += val_loss/len(val_loader)
        print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch+1, epoch_val_accuracy,epoch_val_loss))


Epoch : 1, train accuracy : 0.7194777727127075, train loss : 0.5476154685020447
Epoch : 1, val_accuracy : 0.7670000195503235, val_loss : 0.4899088144302368
Epoch : 2, train accuracy : 0.8034756779670715, train loss : 0.42978644371032715
Epoch : 2, val_accuracy : 0.8125999569892883, val_loss : 0.4204321801662445
Epoch : 3, train accuracy : 0.8315406441688538, train loss : 0.37939396500587463
Epoch : 3, val_accuracy : 0.8014000058174133, val_loss : 0.43539121747016907
Epoch : 4, train accuracy : 0.8533896207809448, train loss : 0.3368593454360962
Epoch : 4, val_accuracy : 0.8043999671936035, val_loss : 0.42775052785873413
Epoch : 5, train accuracy : 0.871738851070404, train loss : 0.2997022271156311
Epoch : 5, val_accuracy : 0.8159997463226318, val_loss : 0.41249415278434753
Epoch : 6, train accuracy : 0.8913886547088623, train loss : 0.2623632252216339
Epoch : 6, val_accuracy : 0.81659996509552, val_loss : 0.4164029657840729
Epoch : 7, train accuracy : 0.9033895134925842, train loss : 0